In [1]:
import itertools
import functools
import math
import operator
import string
import random

import numpy as np

from nltk import tokenize
from nltk.util import ngrams

from scipy.special import expit as sigmoid
# from scipy.stats import wasserstein_distance as earth_movers_dist

In [2]:
import contextlib
import io

with io.StringIO() as str_io, contextlib.redirect_stdout(str_io):
    import this
    zen = str_io.getvalue()

del str_io, this

In [3]:
def distinct_words(text):
    no_punctuation = ''.join(t for t in text if t not in string.punctuation)
    return frozenset(tokenize.word_tokenize(no_punctuation))

text = zen.lower()

# Evolutionary Algorithms
- [Discrete Differential Evolution for Text Summarization](https://www.researchgate.net/publication/281662415_Discrete_Differential_Evolution_for_Text_Summarization)
- [Evolutionary Algorithm for Extractive Text Summarization](https://www.researchgate.net/profile/Ramiz_Aliguliyev/publication/220518077_Evolutionary_Algorithm_for_Extractive_Text_Summarization/links/09e4151356fc2caab6000000.pdf)
- [An Improved Evolutionary Algorithm for Extractive Text Summarization](https://link.springer.com/chapter/10.1007/978-3-642-36543-0_9)

## Sentence Clustering

Let document $D$ be decomposed into a set of $n$ sentences.  
$
D = \{S_1, S_2, ..., S_n\}
$

In [4]:
D = document_sentences = set(tokenize.sent_tokenize(text))

Let terms $T$ be the set of all $m$ distinct words in D.  
$
T = \{t_1, t_2, ..., t_m\}
$

In [5]:
T = document_distinct_words = distinct_words(text)

Let $S_i$ represent the set of distinct terms in sentence $S_i$ with 
$m_i$ distinct terms.  
$
S_i = \{t_1, t_2, ..., t_{m_i}\}
$

In [6]:
S = sentence_distinct_words = {distinct_words(ds) for ds in document_sentences}

In [15]:
text

"the zen of python, by tim peters\n\nbeautiful is better than ugly.\nexplicit is better than implicit.\nsimple is better than complex.\ncomplex is better than complicated.\nflat is better than nested.\nsparse is better than dense.\nreadability counts.\nspecial cases aren't special enough to break the rules.\nalthough practicality beats purity.\nerrors should never pass silently.\nunless explicitly silenced.\nin the face of ambiguity, refuse the temptation to guess.\nthere should be one-- and preferably only one --obvious way to do it.\nalthough that way may not be obvious at first unless you're dutch.\nnow is better than never.\nalthough never is often better than *right* now.\nif the implementation is hard to explain, it's a bad idea.\nif the implementation is easy to explain, it may be a good idea.\nnamespaces are one honking great idea -- let's do more of those!\n"

In [28]:
ngd = normalized_google_distance(text)
ngd('explicit', 'is'), norm_google_distance('explicit', 'is', D, S)

(0.7820114830995408, 0.7820114830995408)

In [ ]:
# seed = None  # need reproducibility option

# N = len(population)
# n = len(sentences)
# k = len(clusters)

# r = range(N)
# s = range(n)

## Similarity Measures

### Jaccard Coefficient Similarity Measure

$
\large
sim_{jaccard}(S_i, S_j) = \frac{|S_i \cap S_j|}{|S_i \cup S_j|}
$

In [48]:
def jaccard_similarity(a, b):
    a, b = set(a), set(b)
    if not a and not b:
        return 1.0
    return len(a & b) / len(a | b)

### Normalized Google Metrics

$
\large
NGD(t_k, t_l) = \frac{max\{log(f_k), log(f_l)\} - log(f_{lk})} {log(n) - min\{log(f_k), log(f_l)\}}
$  

where:
- $t_k$ and $t_l$ are terms 
- $f_k$ is the number of sentences containing $t_k$
- $f_{kl}$ is the number of sentences containing both $t_k$ and $t_l$
- $n$ is the total number of sentences

In [26]:
# double check scientific paper's handling of "bad" log values

def normalized_google_distance(document):
    def ngd(t_k, t_l):
        f_k = sum(t_k in sent for sent in sentence_words)
        f_l = sum(t_l in sent for sent in sentence_words)
        if not (f_k and f_l):
            raise ValueError('terms must be in document')

        f_kl = sum((t_k in sent) and (t_l in sent) for sent in sentence_words)
        if (f_k > 0) and (f_l > 0) and (f_kl == 0):
            return 1.0

        log_kl = (math.log(f_k), math.log(f_l))

        numerator = max(log_kl) - math.log(f_kl)
        denominator = math.log(n) - min(log_kl)
        return numerator / denominator
    
    sentence_words = tuple(frozenset(sent.split()) for sent in tokenize.sent_tokenize(document))
    n = len(sentence_words)
    return ngd


# def norm_google_distance(t_k, t_l, D, S):
#     """Metric for distance between two terms-- tₖ, tₗ"""
    
#     f_k = sum(t_k in sent for sent in S)
#     f_l = sum(t_l in sent for sent in S)
#     if not (f_k and f_l):
#         raise ValueError('terms must be in document')
    
#     f_kl = sum((t_k in sent) and (t_l in sent) for sent in S)
#     if (f_k > 0) and (f_l > 0) and (f_kl == 0):
#         return 1.0
    
#     log_kl = (math.log(f_k), math.log(f_l))
#     n = len(D)
    
#     numerator = max(log_kl) - math.log(f_kl)
#     denominator = math.log(n) - min(log_kl)
#     return numerator / denominator

$
\large 
sim_{NGD}(t_k, t_l) = e^{-NGD(t_k, t_l)}
$

In [ ]:
def norm_google_similarity_term(t_k, t_l, D, S):
    """Metric for similarity between two terms-- tₖ, tₗ"""
    
    ngd = normalized_google_distance(t_k, t_l, D, S)
    return math.exp(-ngd)

$
\large 
sim_{NGD}(S_i, S_j) = \frac{ \sum\limits_{t_k \in S_i} \sum\limits_{t_l \in S_j} sim_{NGD}(t_k, t_l) }{ m_i m_j }
$  

where:
- $S_i$ and $S_j$ are sentences
- $m_i$ is the number of words in $S_i$

In [112]:
def norm_google_similarity_sent(S_i, S_j, D, S):
    total = sum(sum(norm_google_similarity_term(t_k, t_l, D, S) for t_l in S_j) for t_k in S_i)
    return total / len(S_i) / len(S_j)

In [68]:
# if tₖ == tₗ --> 1
assert norm_google_similarity_term('python', 'python', D, S) == 1

# if (tₖ != tₗ) and (fₖ == fₗ == fₖₗ > 0) --> 1
assert norm_google_similarity_term('explicit', 'implicit', D, S) == 1

## Objective Functions

Let $C$ be a partition of D with $k$ clusters.  
$
C = \{C_1, C_2, ..., C_k\}
$

In [ ]:
C = k_clusters = {frozenset(), ...}
#: 1) Two different clusters should have no sentences in common
# assert all(not C_i & C_j for C_i, C_j in itertools.combinations(C, C))

#: 2) Each sentence should definitely be attached to a cluster
# assert functools.reduce(operator.or_, C) == D

#: 3) Each cluster should have at least one sentence assigned
# assert all(C_p for C_p in C)

### Sigmoid Function
$
\large
sigm(x) = \frac{ 1 }{ 1 + e^{-x} }
$

In [46]:
from scipy.special import expit as sigmoid

### Intra-Cluster Similiarity (Cohesion)
$
\large
F_1 = \sum\limits_{p=1}^{k} |C_p| \sum\limits_{S_i, S_j \in C_p} sim(S_i, S_j) \rightarrow max
$

In [6]:
# one paper shows 1/|Cp| and another shows 1/|Cp|^2. Both that show just |Cp| are the two orig ones i found

def cohesion(C, sim):
    total = 0
    for C_p in C:
        for S_i, S_j in itertools.product(C_p, repeat=2):  # should i use itertools.combinations? sim is symmetric
            total += sim(S_i, S_j) / len(C_p)
    return total

### Inter-Cluster Dissimilarity (Separation)
$
\large
F_2 = 
\sum\limits_{p=1}^{k-1} \frac{1}{|C_p|} 
\sum\limits_{q=p+1}^{k} \frac{1}{|C_q|} 
\sum\limits_{S_i \in C_p} 
\sum\limits_{S_l \in C_q} sim(S_i, S_l) 
\rightarrow min
$

In [110]:
def separation(C, sim):
    total = 0
    for C_p, C_q in itertools.combinations(C, r=2):  # this is Σ(p=1 to k-1) and Σ(q=p+1 to k)
        for S_i, S_j in itertools.product(C_p, C_q):    # should i use itertools.combinations?
            total += sim(S_i, S_j) / len(C_p) / len(C_q)  # need D, S args to sim unless partial it in algorithm
    return total

### Criterion Function
$
\large
F = (1 + sigm(F_1))^{F_2} \rightarrow max
$

In [49]:
def criterion_function(C, sim):
    coh = cohesion(C, sim)
    sep = separation(C, sim)
    return pow(1 + sigmoid(coh), sep)

# Modified Discrete Differential Evolution Algorithm

Initialize the population with $N$ chromosomes each composed of $n$ random integers from \[1, k\]. __NOTE:__ $t$ is the iteration step.

$
X_r(t) = [x_{r,1}(t), x_{r,2}(t), ... x_{r,n}(t)]
$  

where:
- $ x_{r,s}(t) \in \{1, 2, ..., k\} $
- $ r = 1, 2, ..., N $
- $ s = 1, 2, ..., n $
- $N$ is the population size
- $n$ is the number of sentences (in the document)
- $k$ is the number of clusters (number of sentences for summary)

In [31]:
random.choices(range(8), k=10)

[7, 1, 2, 2, 5, 3, 0, 0, 1, 5]

In [55]:
def partition(k, n):  #TODO: create more robust algorithm; however this is only run once for initialization
    X_r = []
    while not all(i in X_r for i in range(k)):
        X_r = np.random.choice(range(k), n)
    return X_r
    

def initialize_population(N, k, n):
    assert k <= n
    X = np.array([partition(k, n) for _ in range(N)])
    return X


X = initialize_population(6, 4, 8)
X

array([[2, 0, 3, 1, 2, 2, 2, 3],
       [0, 0, 2, 2, 3, 3, 2, 1],
       [3, 0, 1, 0, 3, 0, 2, 0],
       [1, 2, 1, 3, 0, 2, 0, 0],
       [0, 3, 1, 3, 2, 2, 1, 1],
       [0, 0, 2, 0, 1, 3, 0, 0]])

$
\large
y_{r, s}(t+1) = 
\begin{cases}
    x_{r1,s}(t) + \lambda[x_{r2,s}(t) - x_{r3,s}(t)] & \text{if}\ rand_s < CR \\
    x_{r,s}(t) & \text{otherwise}
\end{cases}
$

where  
- For each $X_r$, randomly sample $X_{r1}(t), X_{r2}(t), X_{r3}(t)$ from the same generation (each distinct)
- $rand_s$ is uniformally distributed random numbers from \[0, 1\] chosen once for each $s \in \{1, 2, ..., n\}$

hyper-parameters 
- $\lambda$ is a scale factor from \[0, 1\]
- $CR$ is the crossover rate from \[0, 1\]

In [121]:
# how to handle new X_r(t+1) clusters to be in [1,...,k]?
def offspring(X, scaling_factor, crossover_rate):
    r, s = X.shape
    Y = np.empty_like(X)
    for i, X_r in enumerate(X):
        choices = tuple(set(range(r)) - {i})
        l, m, n = np.random.choice(choices, size=3, replace=False)
        X_l, X_m,X_n = X[l], X[m], X[n]
        rand = np.random.random_sample(s)  # pull out so it can be used in mutate()
        for j, (x, x_l, x_m, x_n, rand_s) in enumerate(zip(X_r, X_l, X_m, X_n, rand)):
            if rand_s < crossover_rate:
                y = x_l + scaling_factor * (x_m - x_n)
            else:
                y = x
            Y[i, j] = y
    return Y

Y = offspring(X, scaling_factor=0.6, crossover_rate=0.2)
Y

array([[ 2,  0,  1,  1,  4,  2,  2,  3],
       [ 0,  0,  2,  2,  3,  0,  2,  1],
       [ 3,  0,  1,  0,  3,  0,  2,  0],
       [ 1,  2,  1, -1,  0,  1,  0,  0],
       [ 0,  3,  1,  3,  2,  2,  1,  1],
       [ 2,  0,  2,  0,  1,  3,  0,  0]])

$
\large
X_r(t+1) = 
\begin{cases}
    Y_r(t+1) & \text{if}\ f(Y_r(t+1)) > f(X_r(t)) \\
    X_{r}(t) & \text{otherwise}
\end{cases}
$

where 
- $f(\cdot)$ is the objective function to be maximized

In [123]:
def next_generation(X, Y, func):
    next_gen = X.copy()
    Y = offspring(X, scaling_factor, crossover_rate)
    for i, X_r, Y_r in enumerate(zip(X, Y)):
        if func(Y_r) > func(X_r):
            Y[i] = Y_r
    return next_gen

next_generation(X, Y, func=fitness(...))

## Fitness
<img src="./data/pngs/fitness.png" alt="inverse operator psuedo-code" width="33%" align="left"/>

In [111]:
def fitness_1(X_a):
    return cohesion(X_a)

def fitness_2(X_a):
    return 1 / separation(X_a)

def fitness(X_a):
    return criterion_function(X_a)

## Mutation

At each iteration $t+1$ for each $X_r(t)$ creates
$ m_r(t+1) = [m_{r,1}(t), m_{r,2}(t), ..., m_{r,n}(t)] $.  
For each gene, 1 indicates no mutation and 0 means mutate.

$
\large
m_{r,s}(t+1) = 
\begin{cases}
    1 & \text{if}\ rand_s < sigm(y_{r,s}(t+1)) \\
    0 & \text{otherwise}
\end{cases}
$

In [106]:
def mutate(Y, rand):
    M = np.empty_like(Y)
    for i, (y, rand_s) in enumerate(zip(Y.ravel(), rand.ravel())):
        M.ravel()[i] = rand_s < sigmoid(y)

mutate(Y, rand)

## Inversion Operator

### Figure 1 - Psuedo-code
<img src="./data/pngs/fig1_-_inversion_operator_psuedo_code.png" alt="inverse operator psuedo-code" width="33%" align="left"/>

In [43]:
def inversion_operator(X_r, m_r_p1):
    X_r_p1 = X_r.copy()
    S = set(i for i, mutate in enumerate(m_r_p1) if not mutate)
    
    while len(S):
        s_min, s_max = min(S), max(S)
        X_r_p1[s_min] = X_r[s_max]
        X_r_p1[s_max] = X_r[s_min]
        S -= {s_max, s_min}
    
    return X_r_p1

### Figure 2 - Example
<img src="./data/pngs/fig2_-_inversion_operator_diagram.png" alt="inverse operator example" width="33%" align="left"/>

In [44]:
X_r = [3, 2, 4, 2, 3, 1, 4, 1]
m_r_p1 = [0, 1, 1, 0, 1, 0, 0, 1]
X_r_p1 = [4, 2, 4, 1, 3, 2, 3, 1]

assert inversion_operator(X_r, m_r_p1) == X_r_p1

## ROUGE-N
(Recall Oriented Understudy for Gisting Evaluation)

$
\large
ROUGE-N = \frac
{ \sum\limits_{S \in Summ_{ref}} \sum\limits_{N-gram \in S} Count_{match}(N-gram) }
{ \sum\limits_{S \in Summ_{ref}} \sum\limits_{N-gram \in S} Count(N-gram) }
$

In [7]:
def rouge_n(n, y_pred, y_true):
    n_gram_pred = set(ngrams(y_pred, n))
    n_gram_true = set(ngrams(y_true, n))
    return len(n_gram_pred & n_gram_true) / len(n_gram_true)

### testing rouge_n
https://rare-technologies.com/text-summarization-in-python-extractive-vs-abstractive-techniques-revisited/#how_to_evaluate_text

In [12]:
y_true = 'a good diet must have apples and bananas'.split()
y_pred = 'apples and bananas are must for a good diet'.split()

assert rouge_n(1, y_pred, y_true) == 7 / 8
assert rouge_n(2, y_pred, y_true) == 4 / 7

In [50]:
a = "however doctors learned that long inactivity did more harm than good".split()
b = "patients got out of shape developed blood clots and became demoralized".split()

set(a) & set(b)

set()

In [51]:
jaccard_similarity(a, b)

0.0

Preprocess Steps:  
 - stop words
 - lemmatize